In [ ]:
import os
cwd = os.getcwd()
new_cwd = os.path.join(cwd.rsplit("SeinAcoustic_DataAnalysis")[0], "SeinAcoustic_DataAnalysis") if not cwd.endswith("SeinAcoustic_DataAnalysis") else None
if new_cwd != None:
    os.chdir(new_cwd)

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import glob
from IPython.display import display

In [ ]:
from DBBuilder.datareader import Otoriver_DataReader, Sensea_DataReader
from DBBuilder import DBBuilder

In [ ]:
# dataread_otoriver = Otoriver_DataReader(timefreq="h", renamecolumns=True)
filepath_otoriver = "data/Data_SAM/Bougival/input/acoustique/otoriver/otoriver_2021-07-06_2023-12-08.xlsx" 
# df_otoriver = dataread_otoriver.from_input(input=filepath_otoriver)
# display(df_otoriver)

In [ ]:
# dataread_sensea = Sensea_DataReader(timefreq="h")
filepath_sensea = "data/Data_SAM/Bougival/input/acoustique/sensea/sensea_2021-06-16_2023-11-21.csv"
# df_sensea = dataread_sensea.from_input(input=filepath_sensea)
# display(df_sensea)

In [ ]:
# acoustic = pd.merge(df_otoriver, df_sensea, on="date")

In [ ]:
dbbuilder = DBBuilder(output_asdataframe=True, input_timefreq="15min")
output = dbbuilder.build_from_seinAcousticSourceFiles(acoustic_sensea=filepath_sensea, acoustic_otoriver=filepath_otoriver)
for k, df in output["data"].items():
    print(k)
    display(df)

In [ ]:
data_h = dbbuilder.data
data_h["date_h"] = data_h.date.dt.floor("h")
min_max_date = data_h.groupby(["date_h", "suncycle_day", "suncycle_type"])["date"].agg(["min", "max"]).add_prefix("date_agg_").reset_index()
min_max_date
final_df = data_h[['date', 'suncycle_type', 'suncycle_day', 'dist_tw_rising','date_h','dist_tw_setting']]
final_df = final_df.groupby(by=['date_h','suncycle_type', 'suncycle_day'])[['dist_tw_rising', 'dist_tw_setting']].mean()
final_df = pd.merge(min_max_date, final_df, on=['date_h','suncycle_type', 'suncycle_day'])
final_df.head(50)

In [ ]:
final_df[final_df.date_h == final_df[final_df["date_h"].duplicated()].loc[9].date_h]

In [ ]:
dbbuilder.data.head(50)

In [ ]:
dbbuilder.export_to_xlsx(filepath="data/Data_SAM/Bougival/database.xlsx")

In [ ]:
backup = pd.read_excel("data/Data_SAM/Bougival/database.xlsx", sheet_name=None)